# DFA AES Attack

In [1]:
import chipwhisperer as cw

scope = cw.scope()
scope.gain.db = 25
scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 10E6
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "clkgen"

In [2]:
target = cw.target(scope, cw.targets.CW305, fpga_id='35t', force=False)
# target = cw.target(scope, cw.targets.CW305, fpga_id='100t', force=True)

 **make sure the DIP switches on the CW305 board are set as follows**:
- J16 = 1 `route/drive the Artix target from chipwhisperer-lite board clock`
- K16 = 0 `disable return clock from PLLs being outputed to the ChipWhisperer's HS-In channel`

In [3]:
target.vccint_set(1.0)
# disabe all PLLs
target.pll.pll_enable_set(False)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(False, 1)
target.pll.pll_outenable_set(False, 2)


# 1ms is plenty of idling time
target.clkusbautooff = False
target.clksleeptime = 1

In [4]:
# ensure ADC is locked:
scope.clock.reset_adc()
assert (scope.clock.adc_locked), "ADC failed to lock"

In [5]:
target.usb_clk_setenabled(1)

In [6]:
#Basic setup
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.glitch.width = 10
scope.io.hs2 = "glitch"  # output glitch_out on the clock line
scope.io.glitch_lp = False
print(scope.glitch)

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 1
output      = clock_xor



In [7]:
import chipwhisperer.common.results.glitch as glitch
#gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc = glitch.GlitchController(groups=["column0", "column1", "column2", "column3", "other", \
                                     "reset", "normal"], parameters=["width", "offset", "ext_offset", "N"])

gc.set_range("width", 1, 2)
gc.set_range("offset", -6, -5)
gc.set_range("ext_offset", 0, 10)
gc.set_range("N", -5, 5)
gc.set_global_step([0.4])

gc.display_stats()

IntText(value=0, description='column0 count:', disabled=True)

IntText(value=0, description='column1 count:', disabled=True)

IntText(value=0, description='column2 count:', disabled=True)

IntText(value=0, description='column3 count:', disabled=True)

IntText(value=0, description='other count:', disabled=True)

...

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=1.0, continuous_update=False, description='width setting:', disabled=True, max=2.0, min=1.0,…

FloatSlider(value=-6.0, continuous_update=False, description='offset setting:', disabled=True, max=-5.0, min=-…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=-5.0, continuous_update=False, description='N setting:', disabled=True, max=5.0, min=-5.0, r…

In [8]:
print(scope.adc.trig_count)

1977202


In [9]:
scope.glitch.repeat = 1

In [10]:
scope.glitch.output = "clock_xor"

In [11]:
target.vccint_set(1.00)

In [12]:
outputs = []
results = []
obf = []

In [13]:
scope.clock.clkgen_freq = 10E6

In [14]:
from Crypto.Cipher import AES

ktp = cw.ktp.Basic()
key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
# initialize cipher to verify DUT result:
cipher = AES.new(bytes(key), AES.MODE_ECB)

goldciph = list(cipher.encrypt(bytes(text)))

In [15]:
from tqdm import tnrange
import numpy as np
import time
from Crypto.Cipher import AES


project_file = "projects/Tutorial_HW_CW305_DFA.cwp"
project = cw.create_project(project_file, overwrite=True)
traces = []
textin = []
keys = []



#check if a single column is glitched

# 1. check if a single column is fully glitched
# 2. check if that single column is the only glitch
def check_column_glitch(glitched_ct, gold_ct, full_glitch=False):
    column_lookup = [[0, 13, 10, 7], [4, 1, 14, 11], [8, 5, 2, 15], [12, 9, 6, 3]] #shift rows
    all_glitched = True
    for col in range(4):
        all_glitched = True
        for byte in column_lookup[col]:
            if glitched_ct[byte] == gold_ct[byte]:
                all_glitched = False
        if all_glitched:
            break
    if (not all_glitched and (col == 3)):
        return None
    for i in range(4):
        if i == col:
            continue
        for byte in column_lookup[i]:
            if glitched_ct[byte] != gold_ct[byte]:
                return None
    return col

def check_glitch(glitched_ct, gold_ct):
    column_lookup = [[0, 13, 10, 7], [4, 1, 14, 11], [8, 5, 2, 15], [12, 9, 6, 3]] #shift rows
scope.adc.timeout = 0.1

class DelayedKeyboardInterrupt:
    def __enter__(self):
        self.signal_received = False
        self.old_handler = signal.signal(signal.SIGINT, self.handler)

    def handler(self, sig, frame):
        self.signal_received = (sig, frame)
        logging.debug('SIGINT received. Delaying KeyboardInterrupt.')

    def __exit__(self, type, value, traceback):
        signal.signal(signal.SIGINT, self.old_handler)
        if self.signal_received:
            self.old_handler(*self.signal_received)
    
import logging, signal
logging.getLogger().setLevel(logging.ERROR)
for glitch_setting in gc.glitch_values():
    with DelayedKeyboardInterrupt():
        # run aux stuff that should come before trace here
        scope.glitch.offset = glitch_setting[1]
        scope.glitch.width = glitch_setting[0]
        scope.glitch.ext_offset = glitch_setting[2]
        scope.glitch.offset_fine = glitch_setting[3] / 0.4

        ret = cw.capture_trace(scope, target, text, key)
        if not ret:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))
            continue

        if (list(ret.textout) == goldciph):
            gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))
            continue
        else:
            pass
            #print(list(ret.textout))
            #print(goldciph)
            #print(scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset)
        #trace += scope.getLastTrace()
        #check for a glitch in each column of AES
        column_glitches = []
        for column in range(4):
            if check_column_glitch(ret.textout, goldciph, column):
                column_glitches.append(column)



        traces.append(ret.wave)
        project.traces.append(ret)
        outputs.append(list(ret.textout))
        #We're looking for single column glitches here
        x = check_column_glitch(ret.textout, goldciph)
        if not (x is None):
            gc.add("column{}".format(x), (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))
            obf.append(list(ret.textout))
            print(scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset, scope.glitch.offset_fine, x)

        else:
            gc.add("other", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))

-5.859375 1.171875 3 -2 3
-5.859375 1.171875 3 0 3
-5.859375 1.171875 3 0 3
-5.859375 1.171875 3 1 3
-5.859375 1.171875 3 7 3
-5.859375 1.171875 3 -4 3
-5.859375 1.171875 3 -3 3
-5.859375 1.171875 3 2 3
-5.859375 1.171875 3 3 3
-5.859375 1.171875 3 5 3
-5.859375 1.171875 3 7 3
-5.859375 1.171875 3 10 3
-5.859375 1.171875 3 1 3
-5.859375 1.171875 3 4 3
-5.859375 1.171875 3 9 3
-5.859375 1.171875 3 10 3
-5.46875 1.171875 3 -12 3
-5.46875 1.171875 3 -11 3
-5.46875 1.171875 3 -8 3
-5.46875 1.171875 3 -10 3
-5.46875 1.171875 3 -12 3
-5.46875 1.171875 3 -11 3
-5.46875 1.171875 3 -10 3
-5.859375 1.5625 3 0 3
-5.859375 1.5625 3 4 3
-5.859375 1.5625 3 5 3
-5.859375 1.5625 3 6 3
-5.859375 1.5625 3 7 3
-5.859375 1.5625 3 9 3
-5.859375 1.5625 3 -2 3
-5.859375 1.5625 3 1 3
-5.859375 1.5625 3 6 3
-5.859375 1.5625 3 7 3
-5.859375 1.5625 3 9 3
-5.859375 1.5625 3 10 3
-5.859375 1.5625 3 -5 3
-5.859375 1.5625 3 3 3
-5.859375 1.5625 3 4 3
-5.859375 1.5625 3 6 3
-5.859375 1.5625 3 7 3
-5.46875 1.5625 3 -1

In [16]:
project.save()

In [17]:
# import phoenixAES
# r10=phoenixAES.crack_bytes(obf, goldciph, encrypt=True, verbose=1)

In [18]:
import phoenixAES
outputs2=phoenixAES.convert_r8faults_bytes(outputs, goldciph, encrypt=True)
r10=phoenixAES.crack_bytes(outputs2, goldciph, encrypt=True, verbose=2)

f5866f12f27d691da6b685142d4b6d5b: group 0
6dfc6f129e7d69a3a6b6b96d2dadb05b: group 1
6d862a12f23269a364b6b9142dad6dca: group 2
6d866f5af27dbfa3a6a7b91409ad6d5b: group 3
f5866f12f27d691da6b685142d4b6d5b: group 0
6dfc6f129e7d69a3a6b6b96d2dadb05b: group 1
6d862a12f23269a364b6b9142dad6dca: group 2
6d866f5af27dbfa3a6a7b91409ad6d5b: group 3
c8866f12f27d6949a6b6b7142dbd6d5b: group 0
6de96f129a7d69a3a6b6b9912dad875b: group 1
6d86f012f28f69a3fdb6b9142dad6d49: group 2
6d866f81f27d87a3a63eb914c2ad6d5b: group 3
c8866f12f27d6949a6b6b7142dbd6d5b: group 0
6de96f129a7d69a3a6b6b9912dad875b: group 1
6d86f012f28f69a3fdb6b9142dad6d49: group 2
6d866f81f27d87a3a63eb914c2ad6d5b: group 3
1c866f12f27d69cca6b691142d386d5b: group 0
6de96f12bc7d69a3a6b6b9302dadc75b: group 1
6d867d12f28369a363b6b9142dad6da6: group 2
6d866f34f27d79a3a680b914b2ad6d5b: group 3
c8866f12f27d6949a6b6b7142dbd6d5b: group 0
6de96f129a7d69a3a6b6b9912dad875b: group 1
6d86f012f28f69a3fdb6b9142dad6d49: group 2
6d866f81f27d87a3a63eb914c2ad6d5b: 

In [19]:
if r10 is not None:
    from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
    recv_key = key_schedule_rounds(bytearray.fromhex(r10), 10, 0)
    print("AES Key:")
    print(''.join("%02x" % x for x in recv_key))
else:
    print("Sorry, no key found, try another campain, maybe with different parameters...")

AES Key:
2b7e151628aed2a6abf7158809cf4f3c


In [20]:
bytearray(key_schedule_rounds(key, 0, 10))

CWbytearray(b'd0 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6')

In [21]:
bytearray(key)

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')

In [22]:
key = list(key)
assert (recv_key == key), "Failed to recover encryption key\nGot:      {}\nExpected: {}".format(recv_key, key)

In [23]:
scope.dis()
target.dis()